Carga de información a la base de datos

In [184]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import psycopg2 as ps

In [185]:
#importo el csv de las reservas totales de la carpeta de datos
data_reservas = pd.read_csv('datos/reservas_completo.csv', parse_dates=["fecha_reserva", "inicio_estancia","final_estancia"])
data_reservas.info()
data_reservas["id_cliente"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    15000 non-null  datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  object        
 10  precio_noche     14948 non-null  float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        15000 non-null  float64       
 13  ciudad           9828 non-null   object        
dtypes: bool(1), datetime64[ns](3), float64

array(['CL3', 'CL4', 'CL5', ..., 'CL14994', 'CL14997', 'CL14999'],
      dtype=object)

In [210]:
#importo el csv con los eventos de la carpeta de datos y pongo algunas columnas al formato correcto como la columna codigo postal en formato integer
eventos = pd.read_csv('datos/eventos.csv', parse_dates=["fecha_inicio", "fecha_fin"])
#transformo la columna codigo postal a integer
eventos["codigo_postal"] = eventos["codigo_postal"].astype("Int64")
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre         152 non-null    object        
 1   url_evento     152 non-null    object        
 2   codigo_postal  118 non-null    Int64         
 3   direccion      118 non-null    object        
 4   horario        143 non-null    object        
 5   organizacion   122 non-null    object        
 6   fecha_inicio   152 non-null    datetime64[ns]
 7   fecha_fin      152 non-null    datetime64[ns]
dtypes: Int64(1), datetime64[ns](2), object(5)
memory usage: 9.8+ KB


In [187]:
#importo el csv con los clientes de la carpeta de datos y pongo algunas columnas al formato correcto
clientes = pd.read_csv('datos/listado_clientes.csv')
clientes.info()
clientes["ID_cliente_correcto"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14905 entries, 0 to 14904
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   mail                 14905 non-null  object
 1   nombre               14905 non-null  object
 2   apellido             14905 non-null  object
 3   ID_cliente_correcto  14905 non-null  object
dtypes: object(4)
memory usage: 465.9+ KB


0            CL1
1            CL2
2            CL3
3            CL4
4            CL5
          ...   
14900    CL14996
14901    CL14997
14902    CL14998
14903    CL14999
14904    CL15000
Name: ID_cliente_correcto, Length: 14905, dtype: object

In [188]:
#importo el csv con los hoteles completos
hoteles = pd.read_csv('datos/hoteles_completo.csv')
hoteles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      29 non-null     object 
 1   nombre_hotel  29 non-null     object 
 2   competencia   29 non-null     bool   
 3   estrellas     29 non-null     float64
 4   id_ciudad     29 non-null     int64  
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 1.1+ KB


Empezamos con la conexion a la base de datos

In [194]:
#creamos la conexión con la base de datos
conn = ps.connect(
    dbname="Hoteles_AAR_OK",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432",
)

In [195]:
#Creo un cursor para ejecutar las querys
cur = conn.cursor()

In [191]:
#Compruebo que la conexión es correcta
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

In [ ]:
data_insertar = ["Madrid"]
insert_query_ciudad = """ 
                        INSERT INTO ciudad (nombre_ciudad)
                        VALUES (%s) 
"""
# Ejecutamos la query y le indicamos que solo tenemos una ciudad que es Madrid
cur.execute(insert_query_ciudad, data_insertar)
conn.commit()


In [ ]:
# Cerrar la conexión
cur.close()
conn.close()

In [196]:
# vamos a comprobar que se ha creado correctamente
query_ciudad = """
    SELECT * 
    FROM ciudad; 
"""
cur.execute(query_ciudad)
cur.fetchall()

[(1, 'Madrid')]

Ahora cargaremos la tabla eventos

In [197]:
# Sacamos el id de ciudad de la tabla de ciudad
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
dictio_ciudad = dict(cur.fetchall())
dictio_ciudad

{'Madrid': 1}

In [199]:
eventos.columns

Index(['nombre', 'url_evento', 'codigo_postal', 'direccion', 'horario',
       'organizacion', 'fecha_inicio', 'fecha_fin'],
      dtype='object')

In [220]:
eventos.info()
eventos.codigo_postal.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   nombre         152 non-null    object        
 1   url_evento     152 non-null    object        
 2   codigo_postal  118 non-null    Int64         
 3   direccion      118 non-null    object        
 4   horario        143 non-null    object        
 5   organizacion   122 non-null    object        
 6   fecha_inicio   152 non-null    datetime64[ns]
 7   fecha_fin      152 non-null    datetime64[ns]
dtypes: Int64(1), datetime64[ns](2), object(5)
memory usage: 9.8+ KB


<IntegerArray>
[28041, 28039, 28015, 28045,  <NA>, 28050, 28037, 28022, 28042, 28021, 28031,
 28029, 28028, 28026, 28005, 28024, 28011, 28009, 28012, 28004, 28053, 28017,
 28055, 28020, 28007, 28014, 28019, 28030]
Length: 28, dtype: Int64

In [223]:
#Creo la lista que va a contener la información 
data_insertar_eventos = []  

#iteramos sobre el dataframe de eventos para obtener las filas en las que tenemos vacios añadire un codigo defensivo para cambiarlos por None
for i, row in eventos.iterrows():
    nombre_evento = row["nombre"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"] if pd.notna(row["codigo_postal"]) else None
    direccion = row["direccion"] if pd.notna(row["direccion"]) else None 
    horario = row["horario"] if pd.notna(row["horario"]) else None
    fecha_inicio = row["fecha_inicio"]
    fecha_fin = row["fecha_fin"]
    organizacion = row["organizacion"] if pd.notna(row["organizacion"]) else None
    id_ciudad = dictio_ciudad.get("Madrid")

    # incluimos los valores en la lista
    data_insertar_eventos.append((nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad))

data_insertar_eventos

[('Alfabetización Digital para Personas Mayores. ¡Reconectados!',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=03236f16e4315910VgnVCM2000001f4a900aRCRD',
  28041,
  'CALLE LOS BOHEMIOS 1',
  '10:00',
  Timestamp('2025-03-06 00:00:00'),
  Timestamp('2025-03-06 00:00:00'),
  'Centro Sociocultural Bohemios (Villaverde)',
  1),
 ('Amarga victoria',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=bb203bbfeae15910VgnVCM2000001f4a900aRCRD',
  28039,
  'CALLE ALMANSA 9',
  '18:30',
  Timestamp('2025-03-05 00:00:00'),
  Timestamp('2025-03-05 00:00:00'),
  'Centro Sociocultural José de Espronceda (Tetuán)',
  1),
 ('Animación Stop Motion en la Biblioteca San Fermín',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=965e727228cb4910VgnVCM1000001d4a900aRCRD',
  28041,
  'AVENIDA SAN FERMIN 10',
  '18:00',
  Timestamp('202

In [224]:
#creamos la conexión con la base de datos
conn = ps.connect(
    dbname="Hoteles_AAR_OK",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432",
)
#Creo un cursor para ejecutar las querys
cur = conn.cursor()
# Creo la query de inserción de eventos y la ejecuto con los datos que he creado
insert_query_eventos = """ 
                        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario,
                        fecha_inicio, fecha_fin,organizacion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(insert_query_eventos, data_insertar_eventos)
conn.commit()
# Cerrar la conexión
cur.close()
conn.close()

In [ ]:
#creamos la conexión con la base de datos
conn = ps.connect(
    dbname="Hoteles_AAR_OK",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432",
)
#Creo un cursor para ejecutar las querys
cur = conn.cursor()
#Comprobamos que se ha insertado correctamente
query_eventos = """
    SELECT * 
    FROM eventos; 
"""
cur.execute(query_eventos)
cur.fetchmany(5)


[(13,
  'Alfabetización Digital para Personas Mayores. ¡Reconectados!',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=03236f16e4315910VgnVCM2000001f4a900aRCRD',
  28041,
  'CALLE LOS BOHEMIOS 1',
  '10:00',
  datetime.date(2025, 3, 6),
  datetime.date(2025, 3, 6),
  'Centro Sociocultural Bohemios (Villaverde)',
  1),
 (14,
  'Amarga victoria',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=bb203bbfeae15910VgnVCM2000001f4a900aRCRD',
  28039,
  'CALLE ALMANSA 9',
  '18:30',
  datetime.date(2025, 3, 5),
  datetime.date(2025, 3, 5),
  'Centro Sociocultural José de Espronceda (Tetuán)',
  1),
 (15,
  'Animación Stop Motion en la Biblioteca San Fermín',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=965e727228cb4910VgnVCM1000001d4a900aRCRD',
  28041,
  'AVENIDA SAN FERMIN 10',
  '18:00',
  datetime.date(2025, 3, 7

Creo la tabla hoteles. Tener mucho cuidado de no volver a ejecutar los codigos una vez creados para no duplicar 

In [226]:
hoteles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      29 non-null     object 
 1   nombre_hotel  29 non-null     object 
 2   competencia   29 non-null     bool   
 3   estrellas     29 non-null     float64
 4   id_ciudad     29 non-null     int64  
dtypes: bool(1), float64(1), int64(1), object(2)
memory usage: 1.1+ KB


In [231]:
#En la columa de id_hotel debe ser un integer por lo que quitaré las H para que sea un numero y lo convertiré a integer
hoteles["id_hotel"] = hoteles["id_hotel"].str.replace("H", "").astype(int)
hoteles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_hotel      29 non-null     int64  
 1   nombre_hotel  29 non-null     object 
 2   competencia   29 non-null     bool   
 3   estrellas     29 non-null     float64
 4   id_ciudad     29 non-null     int64  
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 1.1+ KB


In [232]:
#En la columa de id_hotel debe ser un integer por lo que quitaré las H para que sea un numero y lo convertiré a integer tambien en el dataframe de reservas
data_reservas["id_hotel"] = data_reservas["id_hotel"].str.replace("H", "").astype(int)
data_reservas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    15000 non-null  datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     14948 non-null  float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        15000 non-null  float64       
 13  ciudad           9828 non-null   object        
dtypes: bool(1), datetime64[ns](3), float64

In [233]:
data_insertar_hoteles = []
for i, row in hoteles.iterrows():
    id_hotel = row["id_hotel"]
    nombre_hotel = row["nombre_hotel"]
    competencia = row["competencia"]
    valoracion = row["estrellas"]
    id_ciudad = dictio_ciudad.get("Madrid")
  
    tupla_hoteles = (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)

    if tupla_hoteles not in data_insertar_hoteles:
        data_insertar_hoteles.append(tupla_hoteles)

data_insertar_hoteles

[(1, 'ibis Styles Madrid Prado', True, 4.7, 1),
 (10, 'ibis budget Madrid Alcala de Henares La Dehesa', True, 4.3, 1),
 (2, 'ibis Madrid Centro las Ventas', True, 4.5, 1),
 (200, 'Gran Hotel Madrid', False, 3.07, 1),
 (201, 'Hotel Brisas del Mar', False, 3.09, 1),
 (202, 'Hotel Camino del Sol', False, 3.07, 1),
 (203, 'Hotel Costa Azul', False, 3.12, 1),
 (204, 'Hotel Encanto Real', False, 3.03, 1),
 (205, 'Hotel Jardines del Rey', False, 2.94, 1),
 (206, 'Hotel Las Estrellas', False, 2.91, 1),
 (207, 'Hotel Los Almendros', False, 3.01, 1),
 (208, 'Hotel Luz de Madrid', False, 3.05, 1),
 (209, 'Hotel Maravilla Real', False, 2.98, 1),
 (210, 'Hotel Mirador Real', False, 2.98, 1),
 (211, 'Hotel Monte Verde', False, 3.1, 1),
 (212, 'Hotel Palacio Imperial', False, 3.01, 1),
 (213, 'Hotel Puerta del Cielo', False, 3.03, 1),
 (214, 'Hotel Rincón Sereno', False, 3.0, 1),
 (215, 'Hotel Sol y Luna', False, 3.01, 1),
 (216, 'Hotel Torre Dorada', False, 2.92, 1),
 (217, 'Hotel Vista Alegre', Fal

In [235]:
#creamos la conexión con la base de datos
conn = ps.connect(
    dbname="Hoteles_AAR_OK",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432",
)
#Creo un cursor para ejecutar las querys
cur = conn.cursor()
#Creamos y ejecutamos la query de inserción de hoteles
insert_query_hoteles = """ 
                        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
                        VALUES (%s, %s, %s, %s, %s)
"""

cur.executemany(insert_query_hoteles, data_insertar_hoteles)
conn.commit()

In [ ]:
# Cerrar la conexión
cur.close()
conn.close()

In [236]:
#Comprobamos que se ha insertado correctamente
query_hoteles = """
    SELECT * 
    FROM hoteles; 
"""
cur.execute(query_hoteles)
cur.fetchmany(5)

[(1, 'ibis Styles Madrid Prado', True, 4.7, 1),
 (10, 'ibis budget Madrid Alcala de Henares La Dehesa', True, 4.3, 1),
 (2, 'ibis Madrid Centro las Ventas', True, 4.5, 1),
 (200, 'Gran Hotel Madrid', False, 3.07, 1),
 (201, 'Hotel Brisas del Mar', False, 3.09, 1)]

Vamos con la tabla clientes

In [237]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14905 entries, 0 to 14904
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   mail                 14905 non-null  object
 1   nombre               14905 non-null  object
 2   apellido             14905 non-null  object
 3   ID_cliente_correcto  14905 non-null  object
dtypes: object(4)
memory usage: 465.9+ KB


In [239]:
data_insertar_clientes = []

for i, row in clientes.iterrows():
    id_cliente = row["ID_cliente_correcto"]
    nombre = row["nombre"]
    apellido = row["apellido"]
    mail = row["mail"]

    tupla_clientes = (id_cliente, nombre, apellido, mail)
    if tupla_clientes not in data_insertar_clientes:
        data_insertar_clientes.append(tupla_clientes)

data_insertar_clientes

[('CL1', 'Feliciana', 'Cantón', 'feliciana.cantón@example.com'),
 ('CL2', 'Leonardo', 'Hierro', 'leonardo.hierro@example.com'),
 ('CL3', 'Maite', 'Calatayud', 'maite.calatayud@example.com'),
 ('CL4', 'Tecla', 'Bonet', 'tecla.bonet@example.com'),
 ('CL5', 'Amílcar', 'Andrés', 'amílcar.andrés@example.com'),
 ('CL6', 'Joan', 'Vazquez', 'joan.vazquez@example.com'),
 ('CL7', 'Chelo', 'Flor', 'chelo.flor@example.com'),
 ('CL8', 'Judith', 'Rivas', 'judith.rivas@example.com'),
 ('CL9', 'Dolores', 'Lobo', 'dolores.lobo@example.com'),
 ('CL10', 'Adolfo', 'Cuervo', 'adolfo.cuervo@example.com'),
 ('CL11', 'Noé', 'Lobo', 'noé.lobo@example.com'),
 ('CL12', 'María Carmen', 'Navarro', 'maría carmen.navarro@example.com'),
 ('CL13', 'Ricardo', 'Caballero', 'ricardo.caballero@example.com'),
 ('CL14', 'Omar', 'Lamas', 'omar.lamas@example.com'),
 ('CL15', 'Luna', 'Segovia', 'luna.segovia@example.com'),
 ('CL16', 'Ágata', 'Pinedo', 'ágata.pinedo@example.com'),
 ('CL17', 'Lázaro', 'Pardo', 'lázaro.pardo@exam

In [240]:
#creamos la conexión con la base de datos
conn = ps.connect(
    dbname="Hoteles_AAR_OK",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432",
)
#Creo un cursor para ejecutar las querys
cur = conn.cursor()
#Creamos la query para insertar los clientes y la ejecutamos
insert_query_clientes = """ 
                        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
                        VALUES (%s, %s, %s, %s)
"""
cur.executemany(insert_query_clientes, data_insertar_clientes)
conn.commit()

In [241]:
#lanzamos la query para comprobar que se ha insertado correctamente
query_clientes = """
    SELECT * 
    FROM clientes; 
"""
cur.execute(query_clientes)
cur.fetchmany(5)

[('CL1', 'Feliciana', 'Cantón', 'feliciana.cantón@example.com'),
 ('CL2', 'Leonardo', 'Hierro', 'leonardo.hierro@example.com'),
 ('CL3', 'Maite', 'Calatayud', 'maite.calatayud@example.com'),
 ('CL4', 'Tecla', 'Bonet', 'tecla.bonet@example.com'),
 ('CL5', 'Amílcar', 'Andrés', 'amílcar.andrés@example.com')]

Vamos con la tabla de reservas

In [243]:
data_reservas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    15000 non-null  datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     14948 non-null  float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        15000 non-null  float64       
 13  ciudad           9828 non-null   object        
dtypes: bool(1), datetime64[ns](3), float64

In [244]:
#creamos el codigo para crear en una lista vacia los datos que vamos a insertar en la tabla reservas
data_insertar_reservas = []
#ahora creamos el codigo para insertar los datos en la tabla reservas
for i, row in data_reservas.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row["inicio_estancia"]
    final_estancia = row["final_estancia"]
    precio_noche = row["precio_noche"]
    id_cliente = row["id_cliente"]
    id_hotel = row["id_hotel"]


    tupla_reservas = (id_reserva, fecha_reserva, inicio_estancia, final_estancia, id_cliente, id_hotel)
    if tupla_reservas not in data_insertar_reservas:
        data_insertar_reservas.append(tupla_reservas)

In [245]:
#creamos la conexión con la base de datos
conn = ps.connect(
    dbname="Hoteles_AAR_OK",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432",
)
#Creo un cursor para ejecutar las querys
cur = conn.cursor()
#Creamos la query para insertar los datos en la tabla reservas y la ejecutamos
insert_query_reservas = """ 
                        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, id_cliente, id_hotel)
                        VALUES (%s, %s, %s, %s, %s, %s)
"""
#ejecutamos la query
cur.executemany(insert_query_reservas, data_insertar_reservas)
conn.commit()

In [246]:
#lanzamos la query para comprobar que se ha insertado correctamente
query_reservas = """
    SELECT * 
    FROM reservas; 
"""
cur.execute(query_reservas)
cur.fetchmany(5)

[('40c4cb55-d1f5-407b-832f-4756b8ff77b4',
  datetime.date(2025, 2, 9),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  None,
  'CL3',
  211),
 ('f2ce8df5-7844-43e1-8c0f-97ce7a208f21',
  datetime.date(2025, 2, 8),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  None,
  'CL4',
  201),
 ('57d4515a-447d-4067-afcb-9bdcf4e4e915',
  datetime.date(2025, 2, 2),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  None,
  'CL5',
  202),
 ('ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7',
  datetime.date(2025, 2, 8),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  None,
  'CL6',
  213),
 ('99c3dc4f-663c-45f7-849a-ac9313f3746a',
  datetime.date(2025, 2, 9),
  datetime.date(2025, 3, 1),
  datetime.date(2025, 3, 2),
  None,
  'CL7',
  204)]